In [1]:
import xarray as xr
import earthkit.data
from earthkit.transforms import aggregate
from dhis2eo.integrations.pandas import dataframe_to_dhis2_json
from dask.distributed import LocalCluster

In [2]:
cluster = LocalCluster()
client = cluster.get_client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 12,Total memory: 96.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:54135,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 12
Started: Just now,Total memory: 96.00 GiB
Comm: tcp://127.0.0.1:54147,Total threads: 3
Dashboard: http://127.0.0.1:54151/status,Memory: 24.00 GiB
Nanny: tcp://127.0.0.1:54138,


In [3]:
pm_file = "../data/pm_final_srilanka_linearp.nc"
pm_ds = xr.open_dataset(pm_file, chunks={"time": 100})
pm_data = pm_ds["__xarray_dataarray_variable__"]

pop_file = "../data/lka_pop_2023_CN_1km_R2025A_UA_v1.tif"
pop_ds = xr.open_dataset(pop_file)
pop_data = pop_ds['band_data'].rename({"x": "lon", "y": "lat"})
pop_aligned = pop_data.interp(lon=pm_data.lon, lat=pm_data.lat, method="nearest")

pm_weighted = pm_data * pop_aligned

district_file = "../data/sri-lanka-provinces.geojson"
features = earthkit.data.from_source("file", district_file)

agg_num = aggregate.spatial.reduce(pm_weighted, features, mask_dim="id")
agg_den = aggregate.spatial.reduce(pop_aligned, features, mask_dim="id")

pw = agg_num / agg_den
pw.name = "pm25_popweighted"

agg_df = pw.to_dataframe().reset_index()

json_dict = dataframe_to_dhis2_json(
    df = agg_df,                    # aggregated pandas.DataFrame
    org_unit_col = 'id',            # column containing the org unit id
    period_col = 'time',            # column containing the period
    value_col = 'pm25_popweighted', # column containing the value
    data_element_id = 'abc123'      # id of the DHIS2 data element
)
# json_dict['dataValues'][:3]

In [4]:
pop_agg = aggregate.spatial.reduce(pop_data, features, how="sum", mask_dim="id")
pop_df = pop_agg.to_dataframe().reset_index()
pop_df

,id,band,spatial_ref,band_data
0,0,1,0,1144432.750
1,1,1,0,1830626.125
2,2,1,0,2791502.500
3,3,1,0,1518770.000
4,4,1,0,2769347.250
5,5,1,0,2158901.000
6,6,1,0,2626631.250
7,7,1,0,1543956.375
8,8,1,0,6238931.500


In [5]:
pop_agg2 = aggregate.spatial.reduce(pop_aligned, features, how="sum", mask_dim="id")
pop_df2 = pop_agg.to_dataframe().reset_index()
pop_df

,id,band,spatial_ref,band_data
0,0,1,0,1144432.750
1,1,1,0,1830626.125
2,2,1,0,2791502.500
3,3,1,0,1518770.000
4,4,1,0,2769347.250
5,5,1,0,2158901.000
6,6,1,0,2626631.250
7,7,1,0,1543956.375
8,8,1,0,6238931.500


In [7]:
import dhis2eo

dhis2eo
variables = {'band_data': 'sum'}
agg = dhis2eo.utils.aggregate.to_org_units(pop_data, features, variables=variables)
print(agg)

AttributeError: module 'dhis2eo.utils' has no attribute 'aggregate'

In [11]:
dhis2eo.utils.aggregate

AttributeError: module 'dhis2eo.utils' has no attribute 'aggregate'